In [20]:
import numpy as np
from dipy.core.gradients import gradient_table
from dipy.data import get_fnames, default_sphere
from dipy.io.gradients import read_bvals_bvecs
from dipy.io.image import load_nifti

This example shows how to use Constrained Spherical Deconvolution (CSD) introduced by Tournier et al. [Tournier2007].

This method is mainly useful with datasets with gradient directions acquired on a spherical grid.

The basic idea with this method is that if we could estimate the response function of a single fiber then we could deconvolve the measured signal and obtain the underlying fiber distribution.

In this way, the reconstruction of the fiber orientation distribution function (fODF) in CSD involves two steps:

Estimation of the fiber response function

Use the response function to reconstruct the fODF

Let’s first load the data. We will use a dataset with 10 b0s and 150 non-b0s with b-value 2000.

In [21]:
hardi_fname, hardi_bval_fname, hardi_bvec_fname = get_fnames('stanford_hardi')

data, affine = load_nifti(hardi_fname)

bvals, bvecs = read_bvals_bvecs(hardi_bval_fname, hardi_bvec_fname)
gtab = gradient_table(bvals, bvecs)

Step 1. Estimation of the fiber response function
There are many strategies to estimate the fiber response function. Here two different strategies are presented.

Strategy 1 - response function estimates from a local brain region One simple way to estimate the fiber response function is to look for regions of the brain where it is known that there are single coherent fiber populations. For example, if we use a ROI at the center of the brain, we will find single fibers from the corpus callosum. The auto_response_ssst function will calculate FA for a cuboid ROI of radii equal to roi_radii in the center of the volume and return the response function estimated in that region for the voxels with FA higher than 0.7.

In [22]:
from dipy.reconst.csdeconv import (auto_response_ssst,
                                   mask_for_response_ssst,
                                   response_from_mask_ssst)

response, ratio = auto_response_ssst(gtab, data, roi_radii=10, fa_thr=0.7)

Note that the auto_response_ssst function calls two functions that can be used separately. First, the function mask_for_response_ssst creates a mask of voxels within the cuboid ROI that meet the FA threshold constraint. This mask can be used to calculate the number of voxels that were kept, or to also apply an external mask (a WM mask for example). Second, the function response_from_mask_ssst takes the mask and returns the response function calculated within the mask. If no changes are made to the mask between the two calls, the resulting responses should be identical.

In [23]:
mask = mask_for_response_ssst(gtab, data, roi_radii=10, fa_thr=0.7)
nvoxels = np.sum(mask)
print(nvoxels)

response, ratio = response_from_mask_ssst(gtab, data, mask)

1254


The response tuple contains two elements. The first is an array with the eigenvalues of the response function and the second is the average S0 for this response.

It is good practice to always validate the result of auto_response_ssst. For this purpose we can print the elements of response and have a look at their values.

The tensor generated from the response must be prolate (two smaller eigenvalues should be equal) and look anisotropic with a ratio of second to first eigenvalue of about 0.2. Or in other words, the axial diffusivity of this tensor should be around 5 times larger than the radial diffusivity.

In [24]:
print(response)
print(ratio)

(array([0.00139919, 0.0003007 , 0.0003007 ]), 416.7372408293461)
0.21491283972219186


We can double-check that we have a good response function by visualizing the response function’s ODF. Here is how you would do that:

In [25]:
from dipy.viz import window, actor
from dipy.sims.voxel import single_tensor_odf

# Enables/disables interactive visualization
interactive = False

scene = window.Scene()
evals = response[0]
evecs = np.array([[0, 1, 0], [0, 0, 1], [1, 0, 0]]).T


response_odf = single_tensor_odf(default_sphere.vertices, evals, evecs)
# transform our data from 1D to 4D
response_odf = response_odf[None, None, None, :]
response_actor = actor.odf_slicer(response_odf, sphere=default_sphere,
                                  colormap='plasma')
scene.add(response_actor)
print('Saving illustration as csd_response.png')
window.record(scene, out_path='csd_response.png', size=(200, 200))
if interactive:
    window.show(scene)

Saving illustration as csd_response.png


In [27]:
#I don't know what that is
scene.rm(response_actor)

Step 2. fODF reconstruction
After estimating a response function, we are ready to start the deconvolution process. Let’s import the CSD model and fit the datasets.

In [28]:
from dipy.reconst.csdeconv import ConstrainedSphericalDeconvModel
csd_model = ConstrainedSphericalDeconvModel(gtab, response)

For illustration purposes we will fit only a small portion of the data. 
Here we visualize a 30x30x20 region:

In [42]:
data_small = data[20:50, 55:85, 20:40]
csd_fit = csd_model.fit(data_small)

100%|██████████| 18000/18000 [00:08<00:00, 2090.22it/s]


In [43]:
print(data.shape)
print(data_small.shape)

(81, 106, 76, 160)
(30, 30, 20, 160)


In [44]:
#Show the CSD-based ODFs also known as FODFs (fiber ODFs)
csd_odf = csd_fit.odf(default_sphere)

In [47]:
#Here we visualize a 30x30x20 region:
interactive = True
fodf_spheres = actor.odf_slicer(csd_odf, sphere=default_sphere, scale=0.9,
                                norm=False, colormap='plasma')

scene.add(fodf_spheres)

print('Saving illustration as csd_odfs.png')
window.record(scene, out_path='csd_odfs.png', size=(600, 600))
if interactive:
    window.show(scene)

Saving illustration as csd_odfs.png


In [48]:
#In DIPY we also provide tools for finding the peak directions (maxima) of the ODFs. For this purpose we recommend using peaks_from_model.
interactive = True
from dipy.direction import peaks_from_model

csd_peaks = peaks_from_model(model=csd_model,
                             data=data_small,
                             sphere=default_sphere,
                             relative_peak_threshold=.5,
                             min_separation_angle=25,
                             parallel=True,
                             num_processes=2)

scene.clear()
fodf_peaks = actor.peak_slicer(csd_peaks.peak_dirs, csd_peaks.peak_values)
scene.add(fodf_peaks)

print('Saving illustration as csd_peaks.png')
window.record(scene, out_path='csd_peaks.png', size=(600, 600))
if interactive:
    window.show(scene)

Saving illustration as csd_peaks.png
